In [3]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain_openai import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import CacheBackedEmbeddings
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

# Initialize ChatOpenAI LLM
llm = ChatOpenAI(temperature=0.1)

# Initialize memory
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

# Initialize file cache
cache_dir = LocalFileStore("./.cache/")

# Split and load documents
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./document.txt")
docs = loader.load_and_split(text_splitter=splitter)

# Initialize embeddings and vectorstore
embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
vectorstore = FAISS.from_documents(docs, cached_embeddings)

# Retrieve context
retriever = vectorstore.as_retriever()

# Define prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. "
            "If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

# Define memory loader
def load_memory(_):
    return memory.load_memory_variables({})["history"]

# Define chain
chain = (
    {
        "context": retriever,
        "history": load_memory,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)

# Define inputs and run the chain
inputs = [
    "Is Aaronson guilty?",
    "What message did he write in the table?",
    "Who is Julia?",
]

for input in inputs:
    result = chain.invoke(input)

    # Save to memory
    memory.save_context(
        {"input": input},
        {"output": result.content},
    )
    
    # Print result
    print(result.content)


According to the text, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.
He wrote "2+2=5" in the dust on the table.
Julia is a character in the text who is associated with Winston.


In [4]:
chain.invoke("list all questions asked")

AIMessage(content='1. Is Aaronson guilty?\n2. What message did he write in the table?\n3. Who is Julia?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 2366, 'total_tokens': 2390, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a613a83e-6a72-407d-95bd-ffd6c9b19659-0', usage_metadata={'input_tokens': 2366, 'output_tokens': 24, 'total_tokens': 2390, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})